In [6]:
pip install --upgrade mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade --user mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade mysql-connector-python


  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/a7/84/b63f11124f808b6f1e3389072bc36cc907929d7574e85f94bf8f18117fe4/mysql_connector_python-8.2.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   - -------------------------------------- 0.7/14.2 MB 20.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.9/14.2 MB 24.0 MB/s eta 0:00:01
   ---------- ----------------------------- 3.7/14.2 MB 29.6 MB/s eta 0:00:01
   ---------------- ----------------------- 5.7/14.2 MB 33.1 MB/s eta 0:00:01
   --------------------- ------------------ 7.7/14.2 MB 34.9 MB/s eta 0:00:01
   --------------------------- ------------ 9.8/14.2 MB 36.6 MB/s eta 0:00:01
   ------------------------------ --------- 10.7/14.2 MB 40.9 MB/s eta 0:00:01
   ----------------------------------- ---- 12.7/14.2 MB 38.5 MB/s eta 0:00:01
   ---------------------------------------  14.2/14.2 MB 38.5 MB/s

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\lee\\AppData\\Local\\Temp\\pip-uninstall-ws8525c2\\libcrypto-3-x64.dll'
Consider using the `--user` option or check the permissions.



In [ ]:
pip install h5py


In [12]:
pip install pymysql


Note: you may need to restart the kernel to use updated packages.


In [8]:
import serial
import pymysql
import mysql.connector
from datetime import datetime  # 날짜 및 시간을 다루기 위한 모듈 추가

# 데이터베이스 연결은 프로그램 시작 시에 열기
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()


ser = serial.Serial('COM3', 115200)  # 시리얼 포트 설정

try:
    for idx in range(30):  # 30번 데이터를 받아옴
        while True:
            data = ser.readline().decode('utf-8').split(',')
            sensor_values = [item.strip() for item in data if item.strip()]
            
            print(f"Received data #{idx + 1}:", sensor_values)
            print(type(sensor_values))

            if len(sensor_values) != 32:  # 센서 값이 26개인지 확인
                print("확인")
                continue

            PostureType = 'CP'
            current_time = datetime.now()  # 현재 날짜 및 시간 가져오기

            try:
                # 센서 값, 자세 분류 및 생성 시간을 데이터베이스에 저장
                cursor.execute("""
                    INSERT INTO trainingdata5
                    (postureType, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, p24, p25, p26,p27,p28,p29,p30,p31,p32)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s) 
                """, [PostureType] + sensor_values)  # 데이터와 자세 분류를 파라미터로 전달
                conn.commit()
                

                print("Data saved to database.")
                break  # 30번의 데이터를 받아온 후 코드 종료
            except mysql.connector.Error as err:
                print(f"Error: {err}")

except KeyboardInterrupt:
    print("Execution terminated by the user.")

# 데이터베이스 연결 닫기
cursor.close()
conn.close()


Received data #1: ['600', '100', '10600', '0', '0', '0', '2900', '6000', '4300', '4800', '4700', '0', '16300', '0', '0', '0', '100', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
<class 'list'>
Data saved to database.
Received data #2: ['700', '100', '10600', '0', '0', '0', '2900', '5900', '4200', '4900', '4600', '0', '16500', '0', '0', '0', '100', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
<class 'list'>
Data saved to database.
Received data #3: ['700', '100', '10700', '0', '0', '0', '2900', '5900', '4300', '4900', '4400', '0', '16600', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
<class 'list'>
Data saved to database.
Received data #4: ['600', '0', '10700', '0', '0', '0', '2900', '5900', '4200', '4900', '4500', '0', '16800', '0', '0', '0', '100', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
<class 'list'>
Data saved to database.
Received data #5

In [43]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [9]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()

# 데이터베이스에서 데이터 불러오기
query = "SELECT * FROM trainingdata"
cursor.execute(query)
data = cursor.fetchall()

# DataFrame으로 변환
columns = ["id", "postureType", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31", "p32", "CREATED_AT"]
df = pd.DataFrame(data, columns=columns)

# 'id' 열 삭제
df = df.drop("id", axis=1)

# 'CREATED_AT' 컬럼 삭제
df = df.drop("CREATED_AT", axis=1)

# X와 y로 나누기
X = df.drop("postureType", axis=1)
y = df["postureType"]

# 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN 모델 학습
knn_model = KNeighborsClassifier(n_neighbors=15, weights='distance')
knn_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_knn = knn_model.predict(X_test)

# 정확도 출력
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN 모델 정확도: {accuracy_knn}")

# 모델 저장
model_filename_knn = "posture_model_knnrepair1.pkl"
joblib.dump(knn_model, model_filename_knn)
print(f"KNN 모델을 {model_filename_knn} 파일로 저장했습니다.")

# 저장된 KNN 모델 불러오기
loaded_model_knn = joblib.load(model_filename_knn)

# 테스트 데이터로 예측
y_loaded_pred_knn = loaded_model_knn.predict(X_test)

# 저장된 KNN 모델의 정확도 출력
loaded_accuracy_knn = accuracy_score(y_test, y_loaded_pred_knn)
print(f"불러온 KNN 모델의 정확도: {loaded_accuracy_knn}")


KNN 모델 정확도: 1.0
KNN 모델을 posture_model_knnrepair1.pkl 파일로 저장했습니다.
불러온 KNN 모델의 정확도: 1.0


In [11]:
from IPython.display import clear_output

import serial
import joblib
import numpy as np

# 아두이노와 연결된 시리얼 포트 설정
ser = serial.Serial('COM3', 115200)

# 저장된 모델 불러오기
model_filename = "posture_model_knnrepair1.pkl"
loaded_model = joblib.load(model_filename)

try:
    while True:
        # 아두이노에서 센서값을 받아옴 (아래 라인은 실제 센서값을 받아오는 코드로 대체되어야 함)
        data = ser.readline().decode('utf-8').split(',')
        sensor_values = [float(item.strip()) for item in data if item.strip()]

        print("Received sensor values:", sensor_values)

        if len(sensor_values) != 32:  # 센서 값이 26개인지 확인
            print("Invalid sensor values. Skipping...")
            continue

        # 입력 데이터를 NumPy 배열로 변환하여 모델에 적용
        input_data = np.array(sensor_values).reshape(1, -1)

        # 모델로 예측
        predicted_label = loaded_model.predict(input_data)[0]
        clear_output(wait=True)
        # 예측된 레이블 출력
        print(f"Predicted label: {predicted_label}")
         
except KeyboardInterrupt:
    print("Execution terminated by the user.")
finally:
    # 시리얼 포트 닫기
    ser.close()


Predicted label: CP
Received sensor values: []
Invalid sensor values. Skipping...


SerialException: ClearCommError failed (PermissionError(13, '장치가 명령을 인식하지 않습니다.', None, 22))

In [35]:
import serial
import pymysql
import mysql.connector
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib

# ... (previous code)

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()

# 데이터베이스에서 데이터 불러오기
query = "SELECT * FROM trainingdata5"
cursor.execute(query)
data = cursor.fetchall()

# DataFrame으로 변환
columns = ["id", "postureType", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31", "p32", "CREATED_AT"]
df = pd.DataFrame(data, columns=columns)

# 'id' 열 삭제
df = df.drop("id", axis=1)

# 'CREATED_AT' 컬럼 삭제
df = df.drop("CREATED_AT", axis=1)

# X와 y로 나누기
X = df.drop("postureType", axis=1)
y = df["postureType"]

# Standard Scaling 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# KNN 모델 학습
knn_model = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_knn = knn_model.predict(X_test)

# 정확도 출력
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN 모델 정확도: {accuracy_knn}")

# 모델 저장
model_filename_knn = "posture_model_knnsc5.pkl"
joblib.dump(knn_model, model_filename_knn)
print(f"KNN 모델을 {model_filename_knn} 파일로 저장했습니다.")

# 스케일러 저장
scaler_filename = "scaler_knnsc5.pkl"
joblib.dump(scaler, scaler_filename)
print(f"스케일러를 {scaler_filename} 파일로 저장했습니다.")

# 저장된 KNN 모델 불러오기
loaded_model_knn = joblib.load(model_filename_knn)

# 저장된 스케일러 불러오기
loaded_scaler = joblib.load(scaler_filename)

# 테스트 데이터 스케일링
X_test_scaled = loaded_scaler.transform(X_test)

# 테스트 데이터로 예측
y_loaded_pred_knn = loaded_model_knn.predict(X_test_scaled)

# 저장된 KNN 모델의 정확도 출력
loaded_accuracy_knn = accuracy_score(y_test, y_loaded_pred_knn)
print(f"불러온 KNN 모델의 정확도: {loaded_accuracy_knn}")


KNN 모델 정확도: 1.0
KNN 모델을 posture_model_knnsc5.pkl 파일로 저장했습니다.
스케일러를 scaler_knnsc5.pkl 파일로 저장했습니다.
불러온 KNN 모델의 정확도: 0.6989247311827957


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [39]:
import serial
import pymysql
import mysql.connector
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler  # Min-Max 스케일러 사용
import joblib

# ... (previous code)

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()

# 데이터베이스에서 데이터 불러오기
query = "SELECT * FROM trainingdata"
cursor.execute(query)
data = cursor.fetchall()

# DataFrame으로 변환
columns = ["id", "postureType", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31", "p32", "CREATED_AT"]
df = pd.DataFrame(data, columns=columns)

# 'id' 열 삭제
df = df.drop("id", axis=1)

# 'CREATED_AT' 컬럼 삭제
df = df.drop("CREATED_AT", axis=1)

# X와 y로 나누기
X = df.drop("postureType", axis=1)
y = df["postureType"]

# Min-Max Scaling 적용
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# KNN 모델 학습
knn_model = KNeighborsClassifier(n_neighbors=16, weights='distance')
knn_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_knn = knn_model.predict(X_test)

# 정확도 출력
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN 모델 정확도: {accuracy_knn}")

# 모델 저장
model_filename_knn = "posture_model_knnmc1.pkl"
joblib.dump(knn_model, model_filename_knn)
print(f"KNN 모델을 {model_filename_knn} 파일로 저장했습니다.")

# 스케일러 저장
scaler_filename = "scaler_knnmc1.pkl"
joblib.dump(scaler, scaler_filename)
print(f"스케일러를 {scaler_filename} 파일로 저장했습니다.")

# 저장된 KNN 모델 불러오기
loaded_model_knn = joblib.load(model_filename_knn)

# 저장된 스케일러 불러오기
loaded_scaler = joblib.load(scaler_filename)

# 테스트 데이터 스케일링
X_test_scaled = loaded_scaler.transform(X_test)

# 테스트 데이터로 예측
y_loaded_pred_knn = loaded_model_knn.predict(X_test_scaled)

# 저장된 KNN 모델의 정확도 출력
loaded_accuracy_knn = accuracy_score(y_test, y_loaded_pred_knn)
print(f"불러온 KNN 모델의 정확도: {loaded_accuracy_knn}")


KNN 모델 정확도: 1.0
KNN 모델을 posture_model_knnmc1.pkl 파일로 저장했습니다.
스케일러를 scaler_knnmc1.pkl 파일로 저장했습니다.
불러온 KNN 모델의 정확도: 0.6666666666666666


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [44]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()

# 데이터베이스에서 데이터 불러오기
query = "SELECT * FROM trainingdata5"
cursor.execute(query)
data = cursor.fetchall()

# DataFrame으로 변환
columns = ["id", "postureType", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31", "p32", "CREATED_AT"]
df = pd.DataFrame(data, columns=columns)

# 'id' 열 삭제
df = df.drop("id", axis=1)

# 'CREATED_AT' 컬럼 삭제
df = df.drop("CREATED_AT", axis=1)

# X와 y로 나누기
X = df.drop("postureType", axis=1)
y = df["postureType"]

# 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_logistic = logistic_model.predict(X_test)

# 정확도 출력
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print(f"로지스틱 회귀 모델 정확도: {accuracy_logistic}")

# 모델 저장
model_filename_logistic = "posture_model_lg5.pkl"
joblib.dump(logistic_model, model_filename_logistic)
print(f"로지스틱 회귀 모델을 {model_filename_logistic} 파일로 저장했습니다.")

# 저장된 로지스틱 회귀 모델 불러오기
loaded_model_logistic = joblib.load(model_filename_logistic)

# 테스트 데이터로 예측
y_loaded_pred_logistic = loaded_model_logistic.predict(X_test)

# 저장된 로지스틱 회귀 모델의 정확도 출력
loaded_accuracy_logistic = accuracy_score(y_test, y_loaded_pred_logistic)
print(f"불러온 로지스틱 회귀 모델의 정확도: {loaded_accuracy_logistic}")


로지스틱 회귀 모델 정확도: 0.9623655913978495
로지스틱 회귀 모델을 posture_model_lg5.pkl 파일로 저장했습니다.
불러온 로지스틱 회귀 모델의 정확도: 0.9623655913978495


C:\Users\lee\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import joblib

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host="project-db-campus.smhrd.com",
    user="ttap",
    password="1234",
    database="ttap",
    port=3307
)

cursor = conn.cursor()

# 데이터베이스에서 데이터 불러오기
query = "SELECT * FROM trainingdata"
cursor.execute(query)
data = cursor.fetchall()

# DataFrame으로 변환
columns = ["id", "postureType", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31", "p32", "CREATED_AT"]
df = pd.DataFrame(data, columns=columns)

# 'id' 열 삭제
df = df.drop("id", axis=1)

# 'CREATED_AT' 컬럼 삭제
df = df.drop("CREATED_AT", axis=1)

# X와 y로 나누기
X = df.drop("postureType", axis=1)
y = df["postureType"]

# 로그 변환
X_log_transformed = X.apply(lambda x: np.log(x + 1))

# Min-Max Scaling 적용
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_log_transformed)

# 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# 로지스틱 회귀 모델 학습
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_logistic = logistic_model.predict(X_test)

# 정확도 출력
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print(f"로지스틱 회귀 모델 정확도: {accuracy_logistic}")

# 모델 저장
model_filename_logistic = "posture_model_lgmm1.pkl"
joblib.dump(logistic_model, model_filename_logistic)
print(f"로지스틱 회귀 모델을 {model_filename_logistic} 파일로 저장했습니다.")

# 스케일러 저장
scaler_filename = "scaler_logistic1.pkl"
joblib.dump(scaler, scaler_filename)
print(f"스케일러를 {scaler_filename} 파일로 저장했습니다.")

# 저장된 로지스틱 회귀 모델 불러오기
loaded_model_logistic = joblib.load(model_filename_logistic)

# 저장된 스케일러 불러오기
loaded_scaler = joblib.load(scaler_filename)

# 테스트 데이터 스케일링
X_test_log_transformed = X_test.apply(lambda x: np.log(x + 1))
X_test_scaled = loaded_scaler.transform(X_test_log_transformed)

# 테스트 데이터로 예측
y_loaded_pred_logistic = loaded_model_logistic.predict(X_test_scaled)

# 저장된 로지스틱 회귀 모델의 정확도 출력
loaded_accuracy_logistic = accuracy_score(y_test, y_loaded_pred_logistic)
print(f"불러온 로지스틱 회귀 모델의 정확도: {loaded_accuracy_logistic}")


로지스틱 회귀 모델 정확도: 1.0
로지스틱 회귀 모델을 posture_model_lgmm1.pkl 파일로 저장했습니다.
스케일러를 scaler_logistic1.pkl 파일로 저장했습니다.


AttributeError: 'numpy.ndarray' object has no attribute 'apply'